# COVID simulation

This notebook simulates the number of COVID cases at Cornell under some simple assumptions using a classic epidemiological model called a compartmental SIR model. It is intended to clarify the role of several important factors in spreading or mitigating the spread of COVID, including:

* classes
* small gatherings (with PPE)
* large parties (without PPE)
* PPE (like masks) and social distancing
* testing

In [1]:
# this cell loads the packages we'll use 
using DataFrames, Plots, Interact
import Plots: plot
plotly()

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-11657593669925786724\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-16160889011614286833\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-11657593669925786724\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Warning: ORCA.jl has been deprecated and all savefig functionality
│ has been implemented directly in PlotlyBase itself.
│ 
│ By implementing in PlotlyBase.jl, the savefig routines are automatically
│ available to PlotlyJS.jl also.
└ @ ORCA /Users/madeleine/.julia/packages/ORCA/U5XaN/src/ORCA.jl:8


Plots.PlotlyBackend()

The next two cells define some useful datatypes: 

* A *group* of students has a size `n`, and a percentage of the group that is susceptible `s`, infected `i` and recovered `r` 
* A *record* remembers the trajectory of infections in each group in the population

In [2]:
# groups 
mutable struct Group
    n::Float64
    s::Float64
    i::Float64
    r::Float64
end

# default constructors
Group(n,s,i) = Group(n,s,i,1-s-i)
Group(n,s) = Group(n,s,1-s,0)

Group

In [3]:
mutable struct Record
    df::DataFrame
end
Record(groups) = Record(DataFrame(i1=groups[1].i, r1=groups[1].r,
                                  i2=groups[2].i, r2=groups[2].r,
                                  i3=groups[3].i, r3=groups[3].r))
record!(record, groups) = push!(record.df, 
    [groups[1].i, groups[1].r, groups[2].i, groups[2].r, groups[3].i, groups[3].r])
function plot(record::Record, groups)
    df = record.df
    max_i = maximum(groups[1].n*df.i1+groups[2].n*df.i2+groups[3].n*df.i3)
    
    p = plot(df.i1, label="group 1 infections", color="blue", linestyle=:dot, legend=:topleft)
    plot!(df.i2, label="group 2 infections", color="green", linestyle=:dot)
    plot!(df.i3, label="group 3 infections", color="red", linestyle=:dot)
    if max_i > 1e-2
        plot!(groups[1].n*df.i1+groups[2].n*df.i2+groups[3].n*df.i3, label="total infections", color="black", linestyle=:dot, marker=:circle)
    else
        plot!(groups[1].n*df.i1+groups[2].n*df.i2+groups[3].n*df.i3, label="total infections", color="black", linestyle=:dot)
    end
    plot!(df.r1, label="group 1 recovered", color="blue")
    plot!(df.r2, label="group 2 recovered", color="green")
    plot!(df.r3, label="group 3 recovered", color="red")
    ylims!(0,1)
    # print("maximum infection rate $(100*round(maximum(groups[1].n*df.i1+groups[2].n*df.i2+groups[3].n*df.i3), digits=3))%")
    p
end

plot (generic function with 4 methods)

The next cell defines the impact of classes, parties, and testing. 

* The probability of getting infected in class is proportional to the infection rate `i` (proportion of classmates currently infected) and the infectivity (which might depend, eg, on PPE and social distancing).
* The probability of getting infected in class is proportional to the probability that *any* attendee is infected, given infection rate `i`, and the infectivity (which might depend, eg, on PPE and social distancing).


In [4]:
##### effect of classes, parties, and testing

"""probability that an susceptible person will be infected in class,
if a proportion i of the class is infected"""
function infected_in_class(i, infectivity)
    return infectivity*i
end

"""probability that an susceptible person will be infected at a party
if a proportion i of the attendees are infected
and prob of infection given any infectious guest is infectivity"""
function infected_at_party(i, party_size, infectivity)
    prob_no_attendee_infectious = (1-i)^party_size
    prob_some_attendee_infectious = 1 - prob_no_attendee_infectious
    return infectivity*prob_some_attendee_infectious
end

"""the fraction of infections caught by test 
and removed from the population (eg, by quarantine)"""
function caught_by_test(i, ttest)
    return i/ttest
end

caught_by_test

The next function simulates how the infections evolve in a population composed of three groups:
   
* people who just attend classes 
* people who also attend small gathering with PPE and social distancing 
* people who also attend large parties without PPE or distancing

The simulation depends on several parameters:
* `n_big_partiers`, the fraction of the population attending big parties with no PPE
* `n_safe_partiers`, the fraction of the population attending small gatherings with PPE
* `ttest`, the frequency of testing: Everyone is tested every `ttest` days
* `inf_no_PPE`, probability of getting infected w/o PPE
* `inf_with_PPE`, probability of getting infected w/PPE
* `ex_i`, the external infection rate (eg, % of NY population infected)
* `party_freq`, how often students attend parties: on average a student attends a party every `party_freq` days
* `legal_party_size`, size of small gatherings
* `illegal_party_size`, size of large parties

In [5]:
function simulate(n_big_partiers = 1e-2,    # big parties, no PPE
    n_safe_partiers = .4,     # small gatherings with PPE
    ttest = 3,                # frequency of testing: test every ttest days
    inf_no_PPE = .1,          # infectivity w/o PPE
    inf_with_PPE = 1e-2,       # infectivity w/PPE
    ex_i = 1e-6,               # external infection rate
    party_freq = 7,             # party_freq = 7 => party once a week
    legal_party_size = 15,
    illegal_party_size = 50
    )
        
    n3 = n_big_partiers
    n2 = n_safe_partiers
    n1 = 1-n3-n2 # only classes
    
    # dynamics
    function update!(groups)
        g1,g2,g3 = groups

        # the proportion of the total population that is infected
        total_i = sum(g.n*g.i for g in groups)
        legal_party_i = (g2.n*g2.i+g3.n*g3.i)/(g2.n+g3.n)

        # update group 1: just classes
        g1.r += caught_by_test(g1.i, ttest)
        g1.i += (g1.s*infected_in_class(total_i,inf_with_PPE)
                + g1.s*ex_i
                - caught_by_test(g1.i,ttest))
        g1.s = 1 - g1.r - g1.i

        # update group 2: small gatherings
        g2.r += caught_by_test(g2.i, ttest)
        g2.i += (g2.s*infected_in_class(total_i,inf_with_PPE)
                + g2.s/party_freq*infected_at_party(legal_party_i,legal_party_size,inf_with_PPE)
                + g2.s*ex_i
                - caught_by_test(g2.i,ttest))
        g2.s = 1 - g2.r - g2.i

        # update group 3: big parties
        g3.r += caught_by_test(g3.i, ttest)
        g3.i += (g3.s*infected_in_class(total_i,inf_with_PPE)
                + g3.s/party_freq*infected_at_party(legal_party_i,legal_party_size,inf_with_PPE)
                + g3.s/party_freq*infected_at_party(g3.i,illegal_party_size,inf_no_PPE)
                + g3.s*ex_i
                - caught_by_test(g3.i,ttest))
        g3.s = 1 - g3.r - g3.i
    end
    
    # initial conditions
    g1 = Group(n1,1)
    g2 = Group(n2,1)
    g3 = Group(n3,1)
    groups = (g1,g2,g3)

    r = Record(groups)
    for t=1:100
        update!(groups)
        record!(r, groups)
    end
    return r, groups
end

simulate (generic function with 10 methods)

Let's simulate from this model to explore influence of parameters. Circle marker shows that the number of infections is large enough that Cornell will close.

In [6]:
@manipulate for n_big_partiers=1e-2:1e-2:.2, n_safe_partiers=.1:.1:1, test_period=1:7, inf_no_PPE=5e-2:5e-2:1, inf_with_PPE=1e-2:1e-2:.1, ex_i = 10. .^ (-7:-1)
    
    legal_party_size = 10
    illegal_party_size = 30
    
    r,groups = simulate(n_big_partiers,    # big parties, no PPE
    n_safe_partiers,     # legal parties with PPE
    test_period,                # frequency of testing: test every ttest days
    inf_no_PPE,          # infectivity w/o PPE
    inf_with_PPE,       # infectivity w/PPE
    ex_i,               # external infection rate
    legal_party_size,
    illegal_party_size
    )
    plot(r, groups)
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["n_big_partiers"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 20,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
10, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/madeleine/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/madeleine/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/madeleine/.julia/packages/InteractBase/sOe2Z/src/../assets/all.js"), Asset("css", nothing, "/Users/madeleine/.julia/packages/InteractBase/sOe2Z/src/../assets/style.css"), Asset("css", nothing, "/Users/madeleine/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000011863cd90, Task (runnable) @0x000000011863cd90), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.01\",\"0.02\",\"0.03\",\"0.04\",\"0.05\",\"0.06\",\"0.07\",\"0.08\",\"0.09\",\"0.1\",\"0.11\",\"0.12\",\"0.13\",\"0.14\",\"0.15\",\"0.16\",\"0.17\",\"0.18\",\"0.19\",\"0.2\"],\"changes\":WebIO.getval({\"name\":\"c

# takeaways from model

* with infrequent testing (weekly+), pandemic is nearly impossible to control
* people who go to big parties get COVID
* if too many people go to big parties, Cornell shuts down
* if PPE is effective,
    * people who just go to classes are ok
    * people who go to small parties might get COVID
* if PPE is not effective,
    * people who just go to classes might get COVID
    * people who go to small parties have even odds of getting COVID